In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class GenerateNoise:
    def __init__(self):
        print("Todo salio bien")
        
        
    def impulsive_noise(self, path_img, noise_percentage):
        self.path_img = path_img
        self.
        pass
    

In [3]:
GenerateNoise()

Todo salio bien
